In [ ]:
pip install cvzone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import cvzone
import math

In [ ]:
#Neu su dung GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [ ]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [ ]:
def loop_rec(frame, keypoints_rec, confidence_threshold):
    for person in keypoints_rec:
        draw_rec(frame, person, confidence_threshold)

def draw_rec(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [ ]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [ ]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [ ]:

192/320

0.6

In [ ]:
def IOU(boxA, boxB):
    # pyimagesearch: determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou
    


In [ ]:
# boxA = (1,2,3,5)
# boxB = (1,2,3,4)
# print(IOU(boxA, boxB))


In [ ]:
class Box:
    def __init__(self, y1=0, x1=0, y2=0, x2=0,conf = 0, count=0):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.conf = conf
        self.count = count

In [ ]:
# box1 = Box(1,2,3,4,5)
# print(box1.x1)

In [ ]:
def update_trackers(boxes1, boxes2):
    A = [] 
    B = []
    C = []
    updated_boxes1 = []
    updated_boxes2 = []
    print('len(boxes1): ',len(boxes1))

    for box2 in boxes2:
        count = 0
        found_match = False
        B[:] = box2.x1,box2.y1, box2.x2, box2.y2
        for box1 in boxes1:
            A[:] = box1.x1,box1.y1, box1.x2, box1.y2
            if IOU(A, B) > 0.6:
                print('IOU',IOU(A, B))
                box2.count = box1.count
                updated_boxes1.append(box1)
                updated_boxes2.append(box2)
                found_match = True
                break
        if not found_match:
            # Thêm các box2 còn lại vào danh sách updated_boxes1
            for box1 in boxes1:
              C[:] = box1.x1,box1.y1, box1.x2, box1.y2
              if IOU(C, B) < 0.5:
                print('IOU_sai: ', IOU(A, B))
                print('box_sai:', box1.count, '-', box2.count)
                count = count + 1
            if count == len(boxes1):
              updated_boxes2.append(box2)
    # Xóa các box1 không được update
    for box1 in boxes1:
        if box1 not in updated_boxes1:
            del box1
    return updated_boxes2


In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/Data-ML-Drive/yolo_v8/video_hau_2.mp4')
# Lấy thông số kích thước video từ video gốc
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
print(frame_width,frame_height)
fps = int(cap.get(5))
fourcc = cv2.VideoWriter_fourcc(*"MP4V")
out = cv2.VideoWriter("/content/drive/MyDrive/Data-ML-Drive/yolo_v8/data_detech/output4.mp4", fourcc, fps, (frame_width, frame_height))
boxes1 = [Box()]
count_frame = 0
count = 0
while True:
  ret,frame = cap.read()

  if not ret:
    break
  #resize:
  count_frame = count_frame+1
  print("frame: ",count_frame)
  img = frame.copy()
  img = tf.image.resize_with_pad(tf.expand_dims(img, axis = 0),480,256)
  input_img = tf.cast(img, dtype = tf.int32)

  #detection
  results = movenet(input_img)
  #print(results)
  keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
  keypoints_rec = results['output_0'].numpy()[:,:,51:].reshape((6,5))
  print('keypoints_rec: ', keypoints_rec)
  print('keypoints_rec[:,-1]: ',keypoints_rec[:,-1])

  boxes2 = []
  for i in range(len(keypoints_rec[:,-1])):
    if keypoints_rec[i,-1] > 0.1:
      y1, x1, y2, x2 = keypoints_rec[i,:4]
      y1, y2, x1, x2 = y1*frame_height, y2*frame_height, x1*frame_width, x2*frame_width
      x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
      w,h = x2 - x1, y2 - y1
      print(x1, y1, x2, y2,w,h)
      cvzone.cornerRect(frame, (x1,y1,w,h))

      conf = keypoints_rec[i,-1]
      conf = math.ceil((conf*100))/100
      count = count + 1
      boxes2.append(Box(y1, x1, y2, x2, conf, count))
  
  for box1 in boxes1:
    print('box1',vars(box1))
  for box2 in boxes2:
    print('box2',vars(box2))
  boxes_f = update_trackers(boxes1, boxes2)
  boxes1 = boxes_f
  for box1 in boxes1:
    print('box1 da update',vars(box1))
    cvzone.putTextRect(frame, f'ID: {box1.count} {box1.conf}', (max(0,box1.x1), max(35, box1.y1)), scale = 3)




  # Render keypoints 
  loop_through_people(frame, keypoints_with_scores, EDGES, 0.5)
  #print(frame.shape)
  #print(input_img.shape)

  out.write(frame)
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:

print(keypoints_with_scores)

[[[4.4437015e-01 6.9113421e-01 5.0913411e-01]
  [4.0452355e-01 7.0512408e-01 6.7294675e-01]
  [4.2167616e-01 6.7782211e-01 5.7389253e-01]
  [4.0919691e-01 7.5136417e-01 6.6952860e-01]
  [4.3768281e-01 6.8285680e-01 6.0003906e-01]
  [5.6865561e-01 8.0833906e-01 8.0570209e-01]
  [5.8356321e-01 6.5567297e-01 8.8084525e-01]
  [7.8224158e-01 8.2679123e-01 6.5799373e-01]
  [8.0428863e-01 6.3151520e-01 6.5097618e-01]
  [9.4828737e-01 8.5634559e-01 7.7902174e-01]
  [9.6392316e-01 6.0645986e-01 5.7237238e-01]
  [9.4508117e-01 7.8585213e-01 7.7444267e-01]
  [9.4722283e-01 6.7925304e-01 7.6098549e-01]
  [9.5022649e-01 8.0533516e-01 5.9253823e-02]
  [9.5691705e-01 6.4404267e-01 2.8864395e-02]
  [9.3000430e-01 7.4167770e-01 8.9181997e-02]
  [9.2276168e-01 7.7044964e-01 7.1094155e-02]]

 [[3.9279377e-01 3.6380139e-01 5.3781980e-01]
  [3.7405699e-01 3.8058484e-01 6.1480594e-01]
  [3.6990413e-01 3.4225354e-01 7.0410085e-01]
  [4.0793556e-01 3.9670607e-01 6.3014543e-01]
  [4.1176423e-01 3.1513309e-01 6